<a href="https://colab.research.google.com/github/sofiaangulodiaz/RECETAS---HE2-PARCIAL-3/blob/main/Codigo_RECETAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalación de librerías

Se hace la instalación de los datasets de [Hugging Face](https://huggingface.co/datasets) y [Sentence Transformers](https://huggingface.co/sentence-transformers). También se instala una LLM de [Ollama](https://huggingface-co.translate.goog/meta-llama/Llama-2-7b?_x_tr_sl=en&_x_tr_tl=es&_x_tr_hl=es&_x_tr_pto=tc).

In [2]:
# Tiempo aproximado de instalación: 2 minutos

# Librerías comunes
import time
import numpy as np
import torch
import subprocess
import json
import requests

# Instalación de datasets de Hugging Face en Colab
!pip uninstall -y datasets huggingface_hub
!pip install "datasets==3.2.0" "huggingface_hub==0.27.0"

# Instalación de los transformers sentence-transformers
!pip install -U "huggingface_hub>=0.34.0" "transformers" "sentence-transformers"

# Instalación de un modelo de Ollama
modelo = "llama2:7b"
!curl -fsSL https://ollama.com/install.sh | sh
!pkill ollama || true
!nohup /usr/local/bin/ollama serve > ollama_output.log 2>&1 &
time.sleep(15)
!curl -s http://localhost:11434/api/tags || echo "El servidor Ollama no está respondiendo"
!ollama pull {modelo}

# Limpia los registros de pip y curl
time.sleep(5)
from IPython.display import clear_output
#clear_output()

Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0
Found existing installation: huggingface-hub 0.36.0
Uninstalling huggingface-hub-0.36.0:
  Successfully uninstalled huggingface-hub-0.36.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.3 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 0.27.0 which is incompatible.
diffusers 0.36.0 requires huggingface-hub<2.0,>=0

# Importar el Dataset

Se carga el dataset de [RecipeNLG](https://huggingface.co/datasets/m3hrdadfi/recipe_nlg_lite). Este dataset contiene 7,198 recetas; por simplicidad se selecciona un subconjunto de 300 recetas.

In [3]:
# Carga el dataset de recetas RecipeNLG
from datasets import load_dataset
print("Cargando RecipeNLG: A Cooking Recipes Dataset...")
dataset = load_dataset("m3hrdadfi/recipe_nlg_lite", trust_remote_code=True) #trust_remote_code=True se usa para evitar el prompt de "run the custom code?(y/N)"
print("\nRecipeNLG Dataset cargado:")
print(dataset)

# Toma un subconjunto de 300 recetas
#train_ds = dataset["train"].shuffle(seed=int(time.time())) #Se copia el dataset y se hace un shuffle para que la selección del subconjunto sea ALEATORIA
train_ds = dataset["train"] #Se copia el dataset (siempre va a tomar las mismas recetas la siguiente línea)
sub_ds = train_ds.select(range(300)) #Toma las primeras 300 recetas (orden aleatorio)
print("\nSe toma un subconjunto de",len(sub_ds),"recetas:")
print(sub_ds)
#print(sub_ds[0])


# Se crea una lista de recetas en formato sencillo (descarta llaves 'link' y 'ner')
recetas = []
for row in sub_ds:
    receta = {
        "id_receta": row["uid"],
        "nombre": row["name"],
        "ingredientes_txt": row["ingredients"],
        "instrucciones": row["steps"],
        "descripcion": row["description"],
    }
    recetas.append(receta)

print("\nEste es un ejemplo de receta, usando formato sencillo:")
#print(recetas[0])
recetas[0]

Cargando RecipeNLG: A Cooking Recipes Dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

recipe_nlg_lite.py: 0.00B [00:00, ?B/s]

dataset_infos.json: 0.00B [00:00, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/6118 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1080 [00:00<?, ? examples/s]


RecipeNLG Dataset cargado:
DatasetDict({
    train: Dataset({
        features: ['uid', 'name', 'description', 'link', 'ner', 'ingredients', 'steps'],
        num_rows: 6118
    })
    test: Dataset({
        features: ['uid', 'name', 'description', 'link', 'ner', 'ingredients', 'steps'],
        num_rows: 1080
    })
})

Se toma un subconjunto de 300 recetas:
Dataset({
    features: ['uid', 'name', 'description', 'link', 'ner', 'ingredients', 'steps'],
    num_rows: 300
})

Este es un ejemplo de receta, usando formato sencillo:


{'id_receta': 'dab8b7d0-e0f6-4bb0-aed9-346e80dace1f',
 'nombre': 'pork chop noodle soup',
 'ingredientes_txt': '3.0 bone in pork chops, salt, pepper, 2.0 tablespoon vegetable oil, 2.0 cup chicken broth, 4.0 cup vegetable broth, 1.0 red onion, 4.0 carrots, 2.0 clove garlic, 1.0 teaspoon dried thyme, 0.5 teaspoon dried basil, 1.0 cup rotini pasta, 2.0 stalk celery',
 'instrucciones': 'season pork chops with salt and pepper . heat oil in a dutch oven over medium high heat . add chops and cook for about 4 minutes, until golden brown . flip and cook 4 minutes more, until golden brown . transfer chops to a plate and set aside . pour half of chicken broth into pot, scraping all browned bits from bottom . add remaining chicken broth, vegetable broth, onion, carrots, celery and garlic . mix well and bring to a simmer . add 1 quart water, thyme, basil, 2 teaspoons salt and 1 teaspoon pepper . mix well and bring to a simmer . add chops back to pot and return to simmer . reduce heat and simmer for

# Encoder

In [4]:
# Crea el encoder y carga el modelo

from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# Encoding de las recetas

# Transforma las recetas (diccionarios) a texto limpio (párrafo con el nombre, ingredientes y pasos)
textos_recetas = []
for r in recetas:
    texto = f"NOMBRE: {r['nombre']}\nINGREDIENTES: {r['ingredientes_txt']}\nPASOS: {r['instrucciones']}"
    textos_recetas.append(texto)


# Codifica cada receta en un vector denso (embedding) de 384 dimensiones
embeddings_recetas = encoder.encode(
    textos_recetas,
    convert_to_numpy=True,
    show_progress_bar=True
)

print("Las",len(textos_recetas),"recetas se han transformado en vectores.")
print("La dimensión final del embedding es:", embeddings_recetas.shape)
print(embeddings_recetas.shape[0],"entradas/recetas han sido codificadas en vectores de",embeddings_recetas.shape[1]," dimensiones.")
print("\nUn ejemplo del encoding:")
print("Texto original:")
print(textos_recetas[0])
print("Embedding:")
print(embeddings_recetas[0])

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Las 300 recetas se han transformado en vectores.
La dimensión final del embedding es: (300, 384)
300 entradas/recetas han sido codificadas en vectores de 384  dimensiones.

Un ejemplo del encoding:
Texto original:
NOMBRE: pork chop noodle soup
INGREDIENTES: 3.0 bone in pork chops, salt, pepper, 2.0 tablespoon vegetable oil, 2.0 cup chicken broth, 4.0 cup vegetable broth, 1.0 red onion, 4.0 carrots, 2.0 clove garlic, 1.0 teaspoon dried thyme, 0.5 teaspoon dried basil, 1.0 cup rotini pasta, 2.0 stalk celery
PASOS: season pork chops with salt and pepper . heat oil in a dutch oven over medium high heat . add chops and cook for about 4 minutes, until golden brown . flip and cook 4 minutes more, until golden brown . transfer chops to a plate and set aside . pour half of chicken broth into pot, scraping all browned bits from bottom . add remaining chicken broth, vegetable broth, onion, carrots, celery and garlic . mix well and bring to a simmer . add 1 quart water, thyme, basil, 2 teaspoons s

# RAG

En esta parte se crean funciones para calcular la similaridad semántica entre un prompt (entrada) y una receta de la base de datos utilizando la distancia coseno.

In [6]:
# Función que calcula la distancia coseno entre dos vectores
def cosine_similarity(a, b):
    a_norm = a / np.linalg.norm(a)
    b_norm = b / np.linalg.norm(b, axis=1, keepdims=True)
    return np.dot(b_norm, a_norm)

# Función que retorna los resultados con mayor simulitud semántica
def buscar_recetas(pregunta, top_k=5):
    emb_q = encoder.encode(pregunta, convert_to_numpy=True) # Hace el encoding del prompt
    sims = cosine_similarity(emb_q, embeddings_recetas) # Obtiene la distancia coseno con todas las recetas del embedding
    top_indices = np.argsort(-sims)[:top_k] # Obtiene los índices de las recetas con mayor similitud
    resultados = []
    for idx in top_indices:
        r = recetas[idx]
        score = float(sims[idx])
        resultados.append((r, score)) # Retorna las recetas (diccionario) con mayor similutud semántica y su distancia coseno
    return resultados

In [7]:
# Ejemplo de consulta

consulta = "Tengo pollo y arroz. Quiero algo fácil tipo sopa, rápido."
# consulta = "I have chicken and rice. I want something easy like soup, fast." # The prompt in english gives the same result as the comparisson is made based on SEMANTIC similarity
resultados = buscar_recetas(consulta, top_k=3) # Busca las 3 recetas con mayor similitud semántica


print("Consulta:",consulta)
print("Resultados:")
print("="*80)
# Muestra la distancia coseno y la receta
for r, score in resultados:
    print("SCORE:", round(score, 3))
    print("NOMBRE:", r["nombre"])
    print("INGREDIENTES:", r["ingredientes_txt"])
    print("PASOS:", r["instrucciones"][:200], "...")
    print("="*80)


Consulta: Tengo pollo y arroz. Quiero algo fácil tipo sopa, rápido.
Resultados:
SCORE: 0.768
NOMBRE: easy and basic chicken child friendly noodle soup recipe
INGREDIENTES: chicken stock, chicken meat, 1.0 tablespoon butter, 1.0 carrot, 0.3 parsnip, clove, sea salt, black pepper, white pepper, dried herbs, soup noodles
PASOS: start with grating your carrot, parsnip and garlic . put the garlic separate since you will lightly butter fry your vegetables and garlic burn easily and gives a bitter taste after that . in your pot  ...
SCORE: 0.685
NOMBRE: fresh pea soup
INGREDIENTES: 1.0 pound peas, 2.0 green onions, 0.1 teaspoon thyme, 1.0 pinch sugar, 2.0 cup chicken broth, 1.0 cup milk, salt, black pepper, nutmeg, carrot
PASOS: place peas, green onion, thyme, sugar and broth in medium saucepan . bring to a boil, reduce heat, cover and simmer 10 minutes . let mixture cool slightly . puree, in batches, in food processor or ble ...
SCORE: 0.679
NOMBRE: beef barley soup
INGREDIENTES: 1 tsp oil, 

CONTEXTO

In [8]:
consulta = "Tengo pollo y arroz. Quiero algo fácil tipo sopa, rápido."
resultados = buscar_recetas(consulta, top_k=3)

for r, score in resultados:
    print("SCORE:", round(score, 3))
    print("NOMBRE:", r["nombre"])
    print("INGREDIENTES:", r["ingredientes_txt"])
    print("PASOS:", r["instrucciones"][:200], "...")
    print("="*80)

SCORE: 0.768
NOMBRE: easy and basic chicken child friendly noodle soup recipe
INGREDIENTES: chicken stock, chicken meat, 1.0 tablespoon butter, 1.0 carrot, 0.3 parsnip, clove, sea salt, black pepper, white pepper, dried herbs, soup noodles
PASOS: start with grating your carrot, parsnip and garlic . put the garlic separate since you will lightly butter fry your vegetables and garlic burn easily and gives a bitter taste after that . in your pot  ...
SCORE: 0.685
NOMBRE: fresh pea soup
INGREDIENTES: 1.0 pound peas, 2.0 green onions, 0.1 teaspoon thyme, 1.0 pinch sugar, 2.0 cup chicken broth, 1.0 cup milk, salt, black pepper, nutmeg, carrot
PASOS: place peas, green onion, thyme, sugar and broth in medium saucepan . bring to a boil, reduce heat, cover and simmer 10 minutes . let mixture cool slightly . puree, in batches, in food processor or ble ...
SCORE: 0.679
NOMBRE: beef barley soup
INGREDIENTES: 1 tsp oil, 1 1/2 lbs lean beef round stew meat, 1 cup chopped carrots, 1 cup chopped onions

# DECODING

Una vez se ha recuperado la información relevante por contenido y contexto, se hace el proceso de construcción de prompt y generación de texto final (decoding).

In [9]:
# Función para construir el prompt a partir de los hits detectados por el RAG
def build_prompt_from_RAG(query):

    hits = buscar_recetas(query, top_k=3) # Busca recetas en la base de datos
    ctx = ""
    for i, cont in enumerate(hits):
      r, score = cont
      ctx += f"Opción de Receta #{i+1}: NOMBRE: {r["nombre"]} INGREDIENTES: {r["ingredientes_txt"]} PASOS: {r["instrucciones"]} \n"

    user = f"Pregunta del Usuario: {query}\n"
    return user + ctx

# Ejemplo
prompt_completo = build_prompt_from_RAG(consulta)
print(prompt_completo)

Pregunta del Usuario: Tengo pollo y arroz. Quiero algo fácil tipo sopa, rápido.
Opción de Receta #1: NOMBRE: easy and basic chicken child friendly noodle soup recipe INGREDIENTES: chicken stock, chicken meat, 1.0 tablespoon butter, 1.0 carrot, 0.3 parsnip, clove, sea salt, black pepper, white pepper, dried herbs, soup noodles PASOS: start with grating your carrot, parsnip and garlic . put the garlic separate since you will lightly butter fry your vegetables and garlic burn easily and gives a bitter taste after that . in your pot add your butter and your grated carrot, parsnip, sea salt, black pepper, white pepper, dries marjoram and stir well until your vegetables become nice and soft . add your grated garlic and stir well up to one minute . now add your chicken stock to the pot and leave to simmer not boil for a little while . in a otter ban boil some thin soup noodles with salt . when there are ready strain them and do not add them to the soup with your chicken meat taken from the bo

In [10]:
# Código tomado del cuaderno de transformers de la Semana 12 https://github.com/dataguirre/Curso-IA-Aplicada/blob/main/Semana%2012_Arquitectura_Transformers/transformers.ipynb

OLLAMA_URL = "http://localhost:11434"
MODEL = "llama2:7b"

# Configuración modificada
SYSTEM_PROMPT = """Eres un chatbot especializado en recomendar recetas de cocina. Espera obtener información como el nombre de un platillo o una lista de ingredientes.
Recibirás también tres o más opciones de recetas a partir de una base de datos.
Debes proporcionar información sobre el nombre de la receta, confirma los ingredientes que se necesitan, y en formato de lista dar los pasos de preparación.
Si el usuario no pregunta por una receta, no le recomiendes opciones, espera a una pregunta directa para ofrecer opciones.
Si el usuario hace preguntas sobre una receta, extiende la explicación de los pasos, pero no ofrezcas otras opciones.
Responde exclusivamente en base a las opciones de recetas obtenidas.
Responde siempre en español traduciendo las recetas cuando sea necesario.
Escribe tu respuesta ajustando mayúsculas y minúsculas en un formato claro y ordenado.
"""

# Funciones para inicializar y reiniciar contexto
def init_context(system_prompt=SYSTEM_PROMPT):
    """Crea un historial nuevo con el mensaje de sistema."""
    return [{"role": "system", "content": system_prompt.strip()}]

def reset_context(context, system_prompt=SYSTEM_PROMPT):
    """Resetea el historial manteniendo el system prompt."""
    context[:] = [{"role": "system", "content": system_prompt.strip()}]
    return context

# --- Llamada al endpoint /api/chat ---
def chat_ollama(prompt, context, model=MODEL, temperature=0.7, max_tokens=1024, seed=None, timeout=120):
    """
    Envía un turno al chat de Ollama con historial.
    - context: lista de dicts [{'role': 'system'|'user'|'assistant', 'content': str}, ...]
    Devuelve: texto de respuesta (str) y actualiza 'context' in-place.
    """
    # Añadir turno del usuario
    context.append({"role": "user", "content": str(prompt)})

    payload = {
        "model": model,
        "messages": context,
        "stream": False,  # respuesta en un único JSON
        "options": {
            "temperature": float(temperature),
            "num_predict": int(max_tokens),
        },
    }
    if seed is not None:
        payload["options"]["seed"] = int(seed)

    url = OLLAMA_URL.rstrip("/") + "/api/chat"
    resp = requests.post(url, json=payload, timeout=timeout)
    resp.raise_for_status()
    data = resp.json()

    # Extraer texto (formato /api/chat o /api/generate)
    text = ""
    if isinstance(data, dict):
        msg = data.get("message")
        if isinstance(msg, dict):
            text = msg.get("content", "")
        if not text:
            text = data.get("response", "")

    # Añadir turno del asistente
    context.append({"role": "assistant", "content": text})
    return text

In [11]:
# Función para preguntarle algo a la LLM
def ask(prompt, context, **kwargs):
    prompt_RAG = build_prompt_from_RAG(prompt) # Primero pasa el prompt por el RAG para obtener opciones de recetas
    text  = chat_ollama(prompt_RAG, context, **kwargs) # Le envía a Olama la pregunta inicial y las opciones
    # Imprime
    print(f'\n {prompt} \n')
    print(text)
    return text

In [12]:
ctx = init_context() # Inicializa contexto (SYSTEM_PROMPT)
ask("Tengo pollo y arroz. Quiero algo fácil tipo sopa, rápido.", ctx)
reset_context(ctx)  # para reiniciar la conversación


 Tengo pollo y arroz. Quiero algo fácil tipo sopa, rápido. 

Hola! Entendido que deseas obtener una receta fácil y rápida para preparar una sopa con pollo y arroz. Aquí te presento tres opciones de recetas que podrían gustarte:

Opción #1: Easy and Basic Chicken Noodle Soup Recipe

Nombre: NOMBRE: easy and basic chicken child friendly noodle soup recipe
INGREDIENTES: chicken stock, chicken meat, 1.0 tablespoon butter, 1.0 carrot, 0.3 parsnip, clove, sea salt, black pepper, white pepper, dried herbs, soup noodles
PASOS:

* Start by grating your carrot and parsnip.
* In a pot, add the butter and grated vegetables and stir until they become soft.
* Add chicken stock to the pot and let it simmer for a few minutes.
* Add dried herbs, salt, black pepper, white pepper, and noodles to the pot.
* Stir well and let it simmer for a few more minutes until the noodles are cooked.
* Serve hot and enjoy!

Opción #2: Fresh Pea Soup

Nombre: NOMBRE: fresh pea soup
INGREDIENTES: 1.0 pound peas, 2.0 gre

[{'role': 'system',
  'content': 'Eres un chatbot especializado en recomendar recetas de cocina. Espera obtener información como el nombre de un platillo o una lista de ingredientes.\nRecibirás también tres o más opciones de recetas a partir de una base de datos.\nDebes proporcionar información sobre el nombre de la receta, confirma los ingredientes que se necesitan, y en formato de lista dar los pasos de preparación.\nSi el usuario no pregunta por una receta, no le recomiendes opciones, espera a una pregunta directa para ofrecer opciones.\nSi el usuario hace preguntas sobre una receta, extiende la explicación de los pasos, pero no ofrezcas otras opciones.\nResponde exclusivamente en base a las opciones de recetas obtenidas.\nResponde siempre en español traduciendo las recetas cuando sea necesario.\nEscribe tu respuesta ajustando mayúsculas y minúsculas en un formato claro y ordenado.'}]

In [13]:
ctx = init_context() # Inicializa contexto (SYSTEM_PROMPT)
ask("Tengo pollo y arroz. Quiero algo fácil tipo sopa, rápido.", ctx)
ask("Si también tengo plátano, ¿qué puedo preparar?", ctx)
reset_context(ctx)  # para reiniciar la conversación


 Tengo pollo y arroz. Quiero algo fácil tipo sopa, rápido. 

Hola! Entiendo que quieres recomendar una receta fácil y rápida para preparar una sopa con pollo y arroz. Basándome en las opciones de recetas que me proporcionaste, te sugiero la segunda opción: "Fresh pea soup".

La razón por la que elegí esta receta es porque es fácil de preparar y no requiere mucho tiempo en el horno. Además, el resultado es una sopa cremosa y deliciosa que combine los sabores frescos de las pepas con el aroma de las hierbas y el sabor salado del pollo.

Para prepararla, necesitarás:

* 1 libra de pepas frescas
* 2 cucharadas de aceite de oliva
* 2 cucharadas de jabón
* 1/2 cucharadita de sal
* 1/4 cucharadita de pimienta
* 6 tazas de caldo de pollo
* 2 tazas de leche
* 1/4 taza de almendras molidas (opcional)

Los pasos para preparar la sopa son los siguientes:

1. En una sartén grande, calienta el aceite de oliva a fuego medio. Agrega las pepas frescas y cocina hasta que estén suaves y translúcidas.
2.

[{'role': 'system',
  'content': 'Eres un chatbot especializado en recomendar recetas de cocina. Espera obtener información como el nombre de un platillo o una lista de ingredientes.\nRecibirás también tres o más opciones de recetas a partir de una base de datos.\nDebes proporcionar información sobre el nombre de la receta, confirma los ingredientes que se necesitan, y en formato de lista dar los pasos de preparación.\nSi el usuario no pregunta por una receta, no le recomiendes opciones, espera a una pregunta directa para ofrecer opciones.\nSi el usuario hace preguntas sobre una receta, extiende la explicación de los pasos, pero no ofrezcas otras opciones.\nResponde exclusivamente en base a las opciones de recetas obtenidas.\nResponde siempre en español traduciendo las recetas cuando sea necesario.\nEscribe tu respuesta ajustando mayúsculas y minúsculas en un formato claro y ordenado.'}]